In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels_new, kernels_reparametrization_space as kernels_repar_space
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2
import torch
import torch.optim as optim
import time

Load monthly data

In [2]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


Load daily data applying max-min ordering

In [22]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []        

for day_index in range(31):
  
    hour_start_index = day_index * 8
    #hour_end_index = (day_index + 1) * 8
    hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    # Load the data for the current day
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        hour_indices, 
        ord_mm=ord_mm,  
        dtype=torch.float 
    )
    # Append the day's data to their respective lists
    daily_aggregated_tensors.append(day_aggregated_tensor)
    daily_hourly_maps.append(day_hourly_map) 

print(daily_aggregated_tensors[0].shape)
print(daily_aggregated_tensors[1][:10])

torch.Size([18126, 4])
tensor([[2.4680e+00, 1.2801e+02, 2.5456e+02, 4.5000e+01],
        [4.9760e+00, 1.3298e+02, 2.6703e+02, 4.5000e+01],
        [4.9760e+00, 1.2303e+02, 2.6393e+02, 4.5000e+01],
        [4.0000e-03, 1.2360e+02, 2.5519e+02, 4.5000e+01],
        [4.0000e-03, 1.3242e+02, 2.5992e+02, 4.5000e+01],
        [4.9760e+00, 1.2983e+02, 2.6550e+02, 4.5000e+01],
        [4.9760e+00, 1.2618e+02, 2.6085e+02, 4.5000e+01],
        [4.0000e-03, 1.2952e+02, 2.6049e+02, 4.5000e+01],
        [4.0000e-03, 1.2650e+02, 2.5915e+02, 4.5000e+01],
        [2.6440e+00, 1.2461e+02, 2.5392e+02, 4.5000e+01]])


# swap columns

In [3]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []      
for day_index in range(31):
    hour_start_index = day_index * 8
    #hour_end_index = (day_index + 1) * 8
    hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data_keep_ori(
    df_map, 
    hour_indices, 
    ord_mm=ord_mm,  # or just omit it
    dtype=torch.float # or just omit it 
    )
    day_aggregated_tensor[:,0], day_aggregated_tensor[:,1] = day_aggregated_tensor[:,4] , day_aggregated_tensor[:,5]

    for map in day_hourly_map.values():
        map[:,0], map[:,1] = map[:,4] , map[:,5]
    daily_aggregated_tensors.append( day_aggregated_tensor )
    daily_hourly_maps.append( day_hourly_map )

print(daily_aggregated_tensors[1][:10])

tensor([[ 2.4617e+00,  1.2802e+02,  2.5456e+02,  4.5000e+01,  2.4617e+00,
          1.2802e+02],
        [ 4.9764e+00,  1.3299e+02,  2.6703e+02,  4.5000e+01,  4.9764e+00,
          1.3299e+02],
        [ 4.9775e+00,  1.2303e+02,  2.6393e+02,  4.5000e+01,  4.9775e+00,
          1.2303e+02],
        [-3.2454e-03,  1.2357e+02,  2.5519e+02,  4.5000e+01, -3.2454e-03,
          1.2357e+02],
        [-6.0702e-04,  1.3241e+02,  2.5992e+02,  4.5000e+01, -6.0702e-04,
          1.3241e+02],
        [ 4.9814e+00,  1.2985e+02,  2.6550e+02,  4.5000e+01,  4.9814e+00,
          1.2985e+02],
        [ 4.9684e+00,  1.2617e+02,  2.6085e+02,  4.5000e+01,  4.9684e+00,
          1.2617e+02],
        [ 6.0990e-03,  1.2955e+02,  2.6049e+02,  4.5000e+01,  6.0990e-03,
          1.2955e+02],
        [ 1.3520e-02,  1.2650e+02,  2.5915e+02,  4.5000e+01,  1.3520e-02,
          1.2650e+02],
        [ 2.6435e+00,  1.2459e+02,  2.5392e+02,  4.5000e+01,  2.6435e+00,
          1.2459e+02]])


Hyper parameters

In [ ]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=100

In [ ]:
# --- End Placeholders ---

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Correct Parameter Initialization (SPATIAL-ONLY) ---
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                
    init_phi1 = init_sigmasq * init_phi2            
    init_phi3 = (init_range_lon / init_range_lat)**2  
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 4-parameter spatial-only list
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )
    ]

    # --- Define learning rates and parameter groups ---
    # 💥 REVISED: Set lr_fast from the global lr
    lr_fast = lr 
    fast_indices = [0, 1, 2, 3] # All parameters
    
    param_groups = [
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    # --- Print Job Info ---
    # Assuming lat_lon_resolution, v, mm_cond_number are defined globally
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Instantiate the Correct Class ---
    model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 REVISED: Call the Optimizer Method with Plateau settings ---
    optimizer, scheduler = model_instance.set_optimizer(
            param_groups,     
            lr=lr,            
            betas=(0.9, 0.99), 
            eps=1e-5, 
            scheduler_type='plateau', # Explicitly set to plateau
            patience=patience,        # Pass patience
            factor=factor             # Pass factor
        )

    # --- Call the Correct Fit Method ---
    out, epoch_ran = model_instance.fit_vecc_scheduler_fullbatch(
            params_list,
            optimizer,
            scheduler, 
            model_instance.matern_cov_SPATIAL_log_reparam, 
            epochs=epochs
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day_idx+1} final results: {out}")

In [ ]:
def cal(a):
    day_indices = [0] 
    for day_idx in day_indices:  

        daily_hourly_map = daily_hourly_maps[day_idx]
        daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

        # Convert initial parameters to a list of 1-element tensors
        params_list = [
            torch.tensor([val], dtype=torch.float64, requires_grad=True, device=device_str) for val in a
        ]
        
        # 💡 CRITICAL: Concatenate the list into a single tensor
        # The full_likelihood function expects a single tensor, not a list
        params_tensor = torch.cat(params_list)

        # Assuming 'kernels_repar_space' has your 'fit_vecchia_adams' class
        model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
                smooth = v,
                input_map = daily_hourly_map,
                aggregated_data = daily_aggregated_tensor,
                nns_map = nns_map,
                mm_cond_number = mm_cond_number,
                nheads = nheads
            )
        
        # 💡 Pass the single 'params_tensor' and the correct 4-parameter spatial covariance function
        bb = model_instance.full_likelihood_avg(
            params = params_tensor, 
            input_data = daily_aggregated_tensor, 
            y = daily_aggregated_tensor[:,2], 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam
        )
     
        cov_map = model_instance.cov_structure_saver(params_tensor, model_instance.matern_cov_SPATIAL_log_reparam)
        vecchia_nll = model_instance.vecchia_space_fullbatch( # Change this to your chosen Vecchia implementation
            params = params_tensor, 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam, 
            cov_map = cov_map # Assuming cov_map is precomputed or computed internally
        )
 
    return bb, vecchia_nll



a = [3.5764, 0.7940, 0.3087, 0.6157]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [3.6764, 0.7940, 0.3087, 0.6157]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")


a = [3.4764, 0.7940, 0.3087, 0.6157]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")


Full Negative Log-Likelihood: (22936.561132578532, 22882.89402724899)
Full Negative Log-Likelihood: (22940.44941805631, 22893.58097275657)
Full Negative Log-Likelihood: (22959.63309778168, 22897.341313474775)
